# 📘 FILE 1-D – GPU, Debugging & Exercises

## 🎯 Mục tiêu

Sau bài này bạn sẽ hiểu:
- **CPU vs GPU** - Khi nào dùng gì
- Cách kiểm tra và sử dụng GPU
- **Device placement** - Chỉ định device cho operations
- **Common errors** - Lỗi newbie thường gặp
- **Debugging tips** - Kỹ thuật debug hiệu quả
- **Best practices** - Quy tắc viết code tốt

---

## 📌 Tại sao phải học về GPU?

- Deep Learning cần **tính toán khổng lồ**
- GPU nhanh hơn CPU **10-100x** cho matrix operations
- Production models **bắt buộc** dùng GPU
- Biết GPU → tiết kiệm thời gian training

---

In [ ]:
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")

---

## 1️⃣ CPU vs GPU

### 🔹 CPU (Central Processing Unit)

**Đặc điểm:**
- 4-16 cores mạnh
- Tốc độ clock cao (3-5 GHz)
- Tối ưu cho **sequential tasks**

**Khi nào dùng:**
- Small models (< 1M parameters)
- Data preprocessing
- Inference với batch size nhỏ

---

### 🔹 GPU (Graphics Processing Unit)

**Đặc điểm:**
- 1000+ cores nhỏ
- Tốc độ clock thấp hơn (~1-2 GHz)
- Tối ưu cho **parallel tasks**

**Khi nào dùng:**
- Large models (> 10M parameters)
- Matrix operations (convolution, matmul)
- Training với batch size lớn

---

### 🔹 So sánh trực quan

```
CPU: [====================] 1 task

GPU: [=][=][=][=][=][=][=][=][=][=]
     [=][=][=][=][=][=][=][=][=][=]  1000+ tasks parallel
     [=][=][=][=][=][=][=][=][=][=]
```

---

---

## 2️⃣ Kiểm tra GPU

### 🔹 Các cách kiểm tra

In [ ]:
# Cách 1: List tất cả physical devices
print("=" * 60)
print("ALL PHYSICAL DEVICES")
print("=" * 60)
devices = tf.config.list_physical_devices()
for device in devices:
    print(f"  {device.device_type:5s}: {device.name}")

# Cách 2: Chỉ GPU
print("\n" + "=" * 60)
print("GPU DEVICES")
print("=" * 60)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for i, gpu in enumerate(gpus):
        print(f"  GPU {i}: {gpu.name}")
else:
    print("  No GPU found")

# Cách 3: Built-in test
print("\n" + "=" * 60)
print("TF BUILT-IN TEST")
print("=" * 60)
print(f"  GPU available: {tf.test.is_built_with_cuda()}")
print(f"  GPU device: {tf.test.gpu_device_name()}")

In [ ]:
# Kiểm tra chi tiết GPU (nếu có)
if gpus:
    gpu_details = tf.config.experimental.get_device_details(gpus[0])
    print("GPU Details:")
    for key, value in gpu_details.items():
        print(f"  {key}: {value}")
else:
    print("No GPU available - running on CPU")
    print("Don't worry! Code vẫn chạy bình thường, chỉ chậm hơn.")

---

## 3️⃣ GPU Memory Management

### 🔹 Vấn đề: GPU memory không đủ

**Mặc định:** TensorFlow chiếm **toàn bộ** GPU memory

**Vấn đề:**
- Không chạy được nhiều process
- Out of Memory (OOM) errors

### 🔹 Giải pháp: Memory Growth

In [ ]:
# Enable memory growth
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled")
        print("TensorFlow sẽ chỉ dùng memory khi cần")
    except RuntimeError as e:
        print(f"Error: {e}")
else:
    print("No GPU available")

In [ ]:
# Giới hạn memory cụ thể (ví dụ: 2GB)
if gpus:
    try:
        # Uncomment để test
        # tf.config.set_logical_device_configuration(
        #     gpus[0],
        #     [tf.config.LogicalDeviceConfiguration(memory_limit=2048)]  # MB
        # )
        print("Memory limit: 2GB (nếu uncomment)")
    except RuntimeError as e:
        print(f"Error: {e}")
else:
    print("No GPU available")

---

## 4️⃣ Device Placement

### 🔹 Tự động vs Thủ công

**Mặc định:** TensorFlow tự chọn device
- GPU nếu có
- CPU nếu không

**Thủ công:** Dùng `tf.device()`

In [ ]:
# Kiểm tra device mặc định
x = tf.constant([[1.0, 2.0], [3.0, 4.0]])
print(f"Default device: {x.device}")

In [ ]:
# Force CPU
with tf.device('/CPU:0'):
    x_cpu = tf.constant([[1.0, 2.0], [3.0, 4.0]])
    y_cpu = tf.matmul(x_cpu, x_cpu)

print(f"x_cpu device: {x_cpu.device}")
print(f"y_cpu device: {y_cpu.device}")

In [ ]:
# Force GPU (nếu có)
if gpus:
    with tf.device('/GPU:0'):
        x_gpu = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        y_gpu = tf.matmul(x_gpu, x_gpu)
    
    print(f"x_gpu device: {x_gpu.device}")
    print(f"y_gpu device: {y_gpu.device}")
else:
    print("No GPU available - skipping GPU test")

### 🔹 Benchmark: CPU vs GPU

In [ ]:
# Function để benchmark
def benchmark_matmul(device_name, matrix_size, num_iterations=100):
    """Benchmark matrix multiplication"""
    with tf.device(device_name):
        # Create random matrices
        a = tf.random.normal([matrix_size, matrix_size])
        b = tf.random.normal([matrix_size, matrix_size])
        
        # Warm up
        _ = tf.matmul(a, b)
        
        # Benchmark
        start = time.time()
        for _ in range(num_iterations):
            c = tf.matmul(a, b)
        end = time.time()
        
    avg_time = (end - start) / num_iterations * 1000  # ms
    return avg_time

# Test với nhiều sizes
sizes = [100, 500, 1000, 2000]
cpu_times = []
gpu_times = []

print("Benchmarking CPU...")
for size in sizes:
    cpu_time = benchmark_matmul('/CPU:0', size, num_iterations=10)
    cpu_times.append(cpu_time)
    print(f"  Size {size:4d}: {cpu_time:6.2f} ms")

if gpus:
    print("\nBenchmarking GPU...")
    for size in sizes:
        gpu_time = benchmark_matmul('/GPU:0', size, num_iterations=10)
        gpu_times.append(gpu_time)
        print(f"  Size {size:4d}: {gpu_time:6.2f} ms")
else:
    print("\nNo GPU available - skipping GPU benchmark")
    gpu_times = [0] * len(sizes)

In [ ]:
# Visualization
if gpus and any(gpu_times):
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    x = np.arange(len(sizes))
    width = 0.35
    plt.bar(x - width/2, cpu_times, width, label='CPU', alpha=0.8)
    plt.bar(x + width/2, gpu_times, width, label='GPU', alpha=0.8)
    plt.xlabel('Matrix Size')
    plt.ylabel('Time (ms)')
    plt.title('CPU vs GPU Performance')
    plt.xticks(x, sizes)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    speedup = [cpu / gpu if gpu > 0 else 0 for cpu, gpu in zip(cpu_times, gpu_times)]
    plt.plot(sizes, speedup, 'go-', linewidth=2, markersize=8)
    plt.xlabel('Matrix Size')
    plt.ylabel('Speedup (CPU time / GPU time)')
    plt.title('GPU Speedup')
    plt.grid(True, alpha=0.3)
    plt.axhline(y=1, color='r', linestyle='--', alpha=0.5, label='No speedup')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\nSpeedup:")
    for size, speed in zip(sizes, speedup):
        print(f"  Size {size:4d}: {speed:.2f}x faster")
else:
    print("No GPU available - skipping visualization")

---

## 5️⃣ Common Errors & Solutions

### ❌ Error 1: Shape Mismatch

In [ ]:
# SAI
try:
    a = tf.constant([[1, 2, 3]])
    b = tf.constant([[1], [2]])
    c = tf.matmul(a, b)
except Exception as e:
    print(f"Error: {e}")
    print(f"\nNguyên nhân: a.shape={a.shape}, b.shape={b.shape}")
    print(f"Matmul yêu cầu: (m, n) @ (n, p) = (m, p)")

In [ ]:
# ĐÚNG
a = tf.constant([[1, 2, 3]])
b = tf.constant([[1], [2], [3]])
c = tf.matmul(a, b)

print(f"a.shape: {a.shape}")
print(f"b.shape: {b.shape}")
print(f"c.shape: {c.shape}")
print(f"Result: {c.numpy()}")

### ❌ Error 2: Dtype Mismatch

In [ ]:
# SAI
try:
    a = tf.constant([1, 2, 3], dtype=tf.int32)
    b = tf.constant([1.0, 2.0, 3.0], dtype=tf.float32)
    c = a + b
except Exception as e:
    print(f"Error: {e}")
    print(f"\nNguyên nhân: a.dtype={a.dtype}, b.dtype={b.dtype}")

In [ ]:
# ĐÚNG - Cách 1: Cast
a = tf.constant([1, 2, 3], dtype=tf.int32)
b = tf.constant([1.0, 2.0, 3.0], dtype=tf.float32)
c = tf.cast(a, tf.float32) + b
print(f"Result: {c.numpy()}")

# ĐÚNG - Cách 2: Dùng cùng dtype từ đầu
a = tf.constant([1.0, 2.0, 3.0], dtype=tf.float32)
b = tf.constant([1.0, 2.0, 3.0], dtype=tf.float32)
c = a + b
print(f"Result: {c.numpy()}")

### ❌ Error 3: Gradient is None

In [ ]:
# SAI
x = tf.constant(2.0)  # Tensor thường

with tf.GradientTape() as tape:
    # QUÊN tape.watch(x)
    y = x ** 2

grad = tape.gradient(y, x)
print(f"Gradient: {grad}")  # None!
print("\nNguyên nhân: Quên watch tensor thường")

In [ ]:
# ĐÚNG - Cách 1: Watch
x = tf.constant(2.0)

with tf.GradientTape() as tape:
    tape.watch(x)
    y = x ** 2

grad = tape.gradient(y, x)
print(f"Gradient: {grad.numpy()}")

# ĐÚNG - Cách 2: Dùng Variable
x = tf.Variable(2.0)

with tf.GradientTape() as tape:
    y = x ** 2

grad = tape.gradient(y, x)
print(f"Gradient: {grad.numpy()}")

### ❌ Error 4: OOM (Out of Memory)

In [ ]:
# VÍ DỤ gây OOM (KHÔNG RUN nếu GPU nhỏ!)
print("Ví dụ code gây OOM:")
print("""
# SAI - Batch size quá lớn
model.fit(X, y, batch_size=10000, epochs=100)

# ĐÚNG - Giảm batch size
model.fit(X, y, batch_size=32, epochs=100)
""")

print("\nCách khắc phục OOM:")
print("1. Giảm batch_size")
print("2. Giảm kích thước model")
print("3. Enable memory growth (đã làm ở trên)")
print("4. Dùng mixed precision training (sẽ học sau)")

### ❌ Error 5: Model not compiled

In [ ]:
# SAI
try:
    model = tf.keras.Sequential([tf.keras.layers.Dense(1)])
    # QUÊN compile
    X = np.random.randn(10, 5).astype(np.float32)
    y = np.random.randn(10, 1).astype(np.float32)
    model.fit(X, y, epochs=1, verbose=0)
except Exception as e:
    print(f"Error: {e}")
    print("\nNguyên nhân: Quên compile model")

In [ ]:
# ĐÚNG
model = tf.keras.Sequential([tf.keras.layers.Dense(1)])
model.compile(optimizer='adam', loss='mse')  # BẮT BUỘC!

X = np.random.randn(10, 5).astype(np.float32)
y = np.random.randn(10, 1).astype(np.float32)
model.fit(X, y, epochs=1, verbose=0)

print("Model trained successfully!")

---

## 6️⃣ Debugging Tips

### 🔹 Tip 1: Print shapes thường xuyên

In [ ]:
# BEST PRACTICE
def debug_shapes(model, sample_input):
    """Print output shape của mỗi layer"""
    print("Layer Shapes:")
    print("=" * 60)
    x = sample_input
    for i, layer in enumerate(model.layers):
        x = layer(x)
        print(f"  Layer {i} ({layer.name:15s}): {x.shape}")
    print("=" * 60)

# Test
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

sample = tf.random.normal([5, 10])  # batch_size=5
debug_shapes(model, sample)

### 🔹 Tip 2: Check for NaN/Inf

In [ ]:
def check_nan_inf(tensor, name="tensor"):
    """Kiểm tra NaN và Inf"""
    has_nan = tf.reduce_any(tf.math.is_nan(tensor))
    has_inf = tf.reduce_any(tf.math.is_inf(tensor))
    
    if has_nan:
        print(f"⚠️  WARNING: {name} contains NaN!")
    if has_inf:
        print(f"⚠️  WARNING: {name} contains Inf!")
    if not has_nan and not has_inf:
        print(f"✓ {name} is clean (no NaN/Inf)")

# Test
good_tensor = tf.constant([1.0, 2.0, 3.0])
bad_tensor = tf.constant([1.0, float('nan'), 3.0])
inf_tensor = tf.constant([1.0, float('inf'), 3.0])

check_nan_inf(good_tensor, "good_tensor")
check_nan_inf(bad_tensor, "bad_tensor")
check_nan_inf(inf_tensor, "inf_tensor")

### 🔹 Tip 3: Visualize gradients

In [ ]:
def plot_gradients(model, X, y):
    """Plot gradient magnitudes"""
    with tf.GradientTape() as tape:
        predictions = model(X)
        loss = tf.keras.losses.mse(y, predictions)
    
    grads = tape.gradient(loss, model.trainable_variables)
    
    # Compute gradient norms
    grad_norms = [tf.norm(g).numpy() for g in grads if g is not None]
    layer_names = [v.name for v in model.trainable_variables]
    
    plt.figure(figsize=(12, 4))
    plt.bar(range(len(grad_norms)), grad_norms)
    plt.xlabel('Layer')
    plt.ylabel('Gradient Norm')
    plt.title('Gradient Magnitudes per Layer')
    plt.xticks(range(len(grad_norms)), [f"L{i}" for i in range(len(grad_norms))], rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("Gradient norms:")
    for i, (name, norm) in enumerate(zip(layer_names, grad_norms)):
        print(f"  {name:30s}: {norm:.6f}")

# Test
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')

X_sample = tf.random.normal([32, 10])
y_sample = tf.random.normal([32, 1])

plot_gradients(model, X_sample, y_sample)

---

## 7️⃣ Best Practices

### ✅ 1. Always normalize input data

In [ ]:
from sklearn.preprocessing import StandardScaler

# BAD
X_bad = np.random.randn(100, 10) * 1000  # Large scale

# GOOD
scaler = StandardScaler()
X_good = scaler.fit_transform(X_bad)

print(f"Before normalization: mean={X_bad.mean():.2f}, std={X_bad.std():.2f}")
print(f"After normalization:  mean={X_good.mean():.2f}, std={X_good.std():.2f}")

### ✅ 2. Use appropriate activation functions

In [ ]:
print("""
CHEAT SHEET:

Task                        Hidden Layers    Output Layer
================================================================
Regression                  ReLU             None (linear)
Binary Classification       ReLU             Sigmoid
Multi-class Classification  ReLU             Softmax
""")

### ✅ 3. Monitor training

In [ ]:
# GOOD - Dùng validation split
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')

X = np.random.randn(1000, 10).astype(np.float32)
y = np.random.randn(1000, 1).astype(np.float32)

history = model.fit(
    X, y,
    epochs=20,
    batch_size=32,
    validation_split=0.2,  # QUAN TRỌNG!
    verbose=0
)

# Plot
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Monitoring')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Kiểm tra:")
print("- Train loss giảm → model đang học")
print("- Val loss giảm → model generalize tốt")
print("- Val loss tăng → overfitting!")

### ✅ 4. Use tf.function for performance

In [ ]:
# Slow - Eager mode
def slow_function(x):
    for i in range(10):
        x = x + 1
    return x

# Fast - Graph mode
@tf.function
def fast_function(x):
    for i in range(10):
        x = x + 1
    return x

# Benchmark
x = tf.random.normal([1000, 1000])

# Slow
start = time.time()
for _ in range(10):
    _ = slow_function(x)
slow_time = time.time() - start

# Fast
start = time.time()
for _ in range(10):
    _ = fast_function(x)
fast_time = time.time() - start

print(f"Slow (eager): {slow_time:.4f}s")
print(f"Fast (graph): {fast_time:.4f}s")
print(f"Speedup: {slow_time/fast_time:.2f}x")

---

## 8️⃣ Final Exercises

### 📝 Exercise 1: Debug Broken Model

Code dưới đây có nhiều lỗi. Tìm và sửa!

In [ ]:
# BROKEN CODE - FIX ME!
"""
# Data
X = np.random.randn(100, 10)
y = np.random.randn(100, 1)

# Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64),  # Bug 1: No activation
    tf.keras.layers.Dense(32),  # Bug 2: No activation
    tf.keras.layers.Dense(1, activation='relu')  # Bug 3: Wrong activation for regression
])

# Train (Bug 4: Not compiled!)
model.fit(X, y, epochs=10)
"""

# YOUR FIXED CODE HERE
# TODO: Fix all bugs

### 📝 Exercise 2: Gradient Debugging

Write function để:
1. Check gradient is None
2. Check gradient contains NaN/Inf
3. Print gradient statistics (min, max, mean)

In [ ]:
# YOUR CODE HERE
def debug_gradient(model, X, y, loss_fn):
    """
    Debug gradients của model
    
    Args:
        model: Keras model
        X: Input data
        y: Target data
        loss_fn: Loss function
    """
    # TODO: Implement
    pass

# Test your function

### 📝 Exercise 3: Build Complete Pipeline

Build end-to-end pipeline:
1. Load data (breast cancer)
2. Normalize
3. Split train/val/test
4. Build model
5. Train với monitoring
6. Evaluate
7. Save model

**Yêu cầu:**
- Accuracy > 95%
- No overfitting
- Clean code với comments

In [ ]:
# YOUR CODE HERE
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# TODO: Complete pipeline

### 📝 Exercise 4: Custom Training Loop (BONUS)

Implement custom training loop với:
- Progress bar
- Learning rate scheduling
- Early stopping
- Gradient clipping

In [ ]:
# YOUR CODE HERE
# TODO: Implement custom training loop

---

## 🎯 Foundation Summary

### ✅ Đã học (Files 1A-1D)

**File 1-A: TensorFlow & Tensor Basics**
- Tensor operations
- Shape, dtype, slicing
- NumPy interop

**File 1-B: Eager Execution & GradientTape**
- Eager execution
- Automatic differentiation
- Manual training loop
- Linear regression from scratch

**File 1-C: tf.keras & First Neural Network**
- Sequential API
- Dense layers
- Activation functions
- model.compile/fit/evaluate/predict

**File 1-D: GPU, Debugging & Exercises**
- CPU vs GPU
- Device placement
- Common errors
- Debugging tips
- Best practices

---

### 🎓 Key Skills Acquired

1. ✅ Hiểu cơ chế TensorFlow
2. ✅ Tính gradient thủ công
3. ✅ Build & train neural networks
4. ✅ Debug & optimize code
5. ✅ GPU awareness

### 📚 Ready for Next Step

**PHẦN 2 - INTERMEDIATE** (Files 2A-2D):
- tf.data pipeline
- Advanced optimizers
- Regularization
- Callbacks & custom training

---

## 📖 References

- [TensorFlow GPU Guide](https://www.tensorflow.org/guide/gpu)
- [TensorFlow Performance Guide](https://www.tensorflow.org/guide/profiler)
- [Debugging TensorFlow](https://www.tensorflow.org/guide/debugging)

---

## 🎉 Chúc mừng!

Bạn đã hoàn thành **FOUNDATION (BEGINNER)**!

**Điều quan trọng:**
- Không phải học thuộc, mà là **hiểu bản chất**
- Practice, practice, practice!
- Quay lại xem lại khi cần

**Next steps:**
1. Làm hết exercises
2. Build 1-2 projects nhỏ
3. Chuyển sang PHẦN 2

---

**Chúc bạn học tốt! 🚀**